In [15]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats._, cats.implicits._
import cats.effect.IO
import fs2.Stream

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats._, cats.implicits._

import cats.effect.IO

import fs2.Stream

# Variation 5. DAO in Scala

This is not DAO-based code. No interfaces for now.

Common case classes for DAO interfaces.

In [16]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

defined class Country
defined class City

### IO

In [17]:
class CityQueriesIO(xa: Transactor[IO]){
    def getCityName(id: Int): IO[Option[String]] = 
        sql"select name from city where id = $id"
            .query[String].option.transact(xa)
    
    def getCityPopulation(id: Int): IO[Option[Int]] = 
        sql"select population from city where id = $id"
            .query[Int].option.transact(xa)
}

object CityQueriesIO extends CityQueriesIO(xa)

class CountryQueriesIO(xa: Transactor[IO]){
    def getAllCountries: IO[List[Country]] = 
        sql"select code, name, capital from country"
            .query[Country].to[List].transact(xa)
    
    def getCountryName(id: Int): IO[Option[String]] =
        sql"select name from country where id = $id"
            .query[String].option.transact(xa)
    
    def getCountryCapital(id: Int): IO[Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Int].option.transact(xa)
}

object CountryQueriesIO extends CountryQueriesIO(xa)

defined class CityQueriesIO
defined object CityQueriesIO
defined class CountryQueriesIO
defined object CountryQueriesIO

No filter for IO, can't write our query easily. TBD.

In [23]:
val largeCapitalsIO: IO[List[Option[(String, String)]]] = for {
    countries <- CountryQueriesIO.getAllCountries
    result <- countries.traverse{
        case Country(code,name,Some(capital)) => 
            CityQueriesIO.getCityName(capital).map(_.map((_, name)))
        case _ => None.pure[IO]
    }
} yield result

largeCapitalsIO: IO[List[Option[(String, String)]]] = Bind(
  Async(
    cats.effect.internals.IOBracket$$$Lambda$3320/1660811820@4325d654,
    false
  ),
  ammonite.$sess.cmd22$Helper$$Lambda$3694/2028445427@659913ca
)

In [24]:
largeCapitalsIO.unsafeRunSync.timed

3016 millis


res23: List[Option[(String, String)]] = List(
  Some(("Kabul", "Afghanistan")),
  Some(("Amsterdam", "Netherlands")),
  Some(("Willemstad", "Netherlands Antilles")),
  Some(("Tirana", "Albania")),
  Some(("Alger", "Algeria")),
  Some(("Fagatogo", "American Samoa")),
  Some(("Andorra la Vella", "Andorra")),
  Some(("Luanda", "Angola")),
  Some(("The Valley", "Anguilla")),
  Some(("Saint John\u00b4s", "Antigua and Barbuda")),
  Some(("Abu Dhabi", "United Arab Emirates")),
  Some(("Buenos Aires", "Argentina")),
  Some(("Yerevan", "Armenia")),
  Some(("Oranjestad", "Aruba")),
  Some(("Canberra", "Australia")),
  Some(("Baku", "Azerbaijan")),
  Some(("Nassau", "Bahamas")),
  Some(("al-Manama", "Bahrain")),
  Some(("Dhaka", "Bangladesh")),
  Some(("Bridgetown", "Barbados")),
  Some(("Bruxelles [Brussel]", "Belgium")),
  Some(("Belmopan", "Belize")),
  Some(("Porto-Novo", "Benin")),
  Some(("Hamilton", "Bermuda")),
  Some(("Thimphu", "Bhutan")),
  Some(("La Paz", "Bolivia")),
  Some(("Sarajev

BIG BIG BIG PROBLEM: no filters AND query avalance AND multiple connections

### `Stream[ConnectionIO, ?]`

In [25]:
object CityQueries{
    def getCityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def getCityPopulation(id: Int): Stream[ConnectionIO,Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream
}

object CountryQueries{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(id: Int): Stream[ConnectionIO,String] =
        sql"select name from country where id = $id"
            .query[String].stream
    
    def getCountryCapital(id: Int): Stream[ConnectionIO, Int] =
        sql"select capital from country where id = $id"
            .query[Int].stream
}

import CityQueries._, CountryQueries._

defined object CityQueries
defined object CountryQueries
import CityQueries._, CountryQueries._

In [26]:
val largeCapitals: Stream[ConnectionIO,(String, String)] = for {
    Country(_, name, Some(capital)) <- getAllCountries
    population <- getCityPopulation(capital)
    if population > 8000000
    cityName <- getCityName(capital)
} yield (cityName, name)

largeCapitals: Stream[ConnectionIO, (String, String)] = Stream(..)

In [27]:
largeCapitals.compile
    .toList
    .transact(xa)
    .unsafeRunSync
    .timed

736 millis


res26: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

BIG PROBLEM: query avalanche (although single connection).